## The Vanishing and Exploding Gradients Problems
We need the variance of the output of the layers to be equal to the variance of the inputs

Sigmoid is an example of saturating activation function

Glorot Initialization
fan_avg = (fan_in + fan_out)/2

RELU suffers from a problem called dying RELU's. This is when the weighted sum of outputs for a nueron goes negative and it effectively outputs 0
To counter this you can use Leaky RELU

ELU or exponential linear unit outperfoms RELU
It is slower to compute

Scaled ELU gurantess self-normalization under certain conditions


## Batch Normalization
This operation is added just before or after the activation of each layer.
This operation zero-centers and normalizes each input
There are two new parameter vectors, one for scaling and one for shifting


It calculates the mean and sd over each mini Batch
lambda the scale vector and beta the offset vector are learned through back prop

Batch Normalization also acts as regularizer

In [ ]:
from tensorflow import keras
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    # This gives the same affect as standard scaler
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation='elu', kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation='elu', kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation='softmax')
])


## Gradient Clipping
This is to clip the gradients in the backprop to some max threshold
This is often used for RNN as Batch Norm is tricky to use in RNN

In [ ]:
optimizer = keras.optimizers.SGD(clipvalue=1.0)
model.compile(loss='mse', optimizer=optimizer)

# Another option is to use clip norm

## Transfer Learning

In [ ]:
# Model A is created on 8 classes of Fashion Mnist data set
# Want to train a binary classifier which is model B
model_A = keras.models.load_model("my_model_A.h5")
# Take all but the last layer
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

# Training model B will also affect the weights of Model A
# You will need to clone Model A using clone.model() and then copy its weights

model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

# Initially there will be large errors in the last layer that might get propogated to the to first layers
# We might want to freeze those layers for first few epochs

for layer in model_B_on_A.layers[:-1]
    layer.trainable = False

model_B_on_A.compile(loss='binary_crossentropy', optimizer='sgd', metrics=["accuracy"])
# You must always compile the model after you freeze or unfreeze

# After few epochs, you can unfreeze the model, recompile the model and probably lower the learning rate to avoid 
# damaging the learned rates for earlier layers

history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4, validation_data=(X_valid_B, y_valid_B))

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

optimizer = keras.optimizers.SGD(lr=1e-4)
model_B_on_A.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16, validation_data=(X_valid_B, y_valid_B))

# transfer learning does not work very well with small dense networks. small patterns learn very few patterns
# and dense networks learn very specific patterns

